In [1]:
!pip install sympy -U

In [2]:
import sympy as sym
import control as ctrl
import matplotlib.pyplot as plt
import numpy as np

# Define constants used
m, g, d, delta, R, L_0, L_1, alpha, c, k, b, phi = sym.symbols('m, g, d, delta, R, L_0, L_1, alpha, c, k, b, phi', positive=True)
# Define SSR state variables and inputs
V, x1, x2, x3 = sym.symbols('V, x1, x2, x3', real=True)

In [3]:
# theta(x1, x2, x3)
theta = 5 / (7 * m) * ((c * (x3 ** 2) / (delta - x1) ** 2) + m * g * sym.sin(phi) - k * (x1 - d) - b * x2)
theta

5*(-b*x2 + c*x3**2/(delta - x1)**2 + g*m*sin(phi) - k*(-d + x1))/(7*m)

In [4]:
# Inductance 
L = L_0 + L_1 * sym.exp(-alpha * (delta - x1))

# I_dot(x1, x3, V)
I_dot = (V - x3 * R) / L
I_dot

(-R*x3 + V)/(L_0 + L_1*exp(-alpha*(delta - x1)))

In [5]:
# Equilibrium points calculated in equilibrium_points.ipynb

# x1 is the position of the ball, we are aiming to keep it around 0.5 m
x1_sp = 0.5
# x2 is the velocity of the ball, we are aiming to keep it around 0 m/s
x2_sp = 0
# Current
x3_sp = 0.707488266645418
# Voltage
V_sp = 1556.47418661991

In [6]:
# Parameters provided to us and calculated set-points
PARAMETERS = [
    (m, 0.462),
    (g, 9.81),
    (d, 0.42),
    (delta, 0.65),
    (R, 2200),
    (L_0, 0.125),
    (L_1, 0.0241),
    (alpha, 1.2), # changed from 1/1.2 to preserve m^-1 unit
    (c, 0.006811), # changed from 6.811 to change unit to m^3 kg/ A^2 s^2
    (k, 1885),
    (b, 10.4),
    (phi, 41),
    (x1, x1_sp),
    (x2, x2_sp),
    (x3, x3_sp),
    (V, V_sp)
]

In [7]:
# The value of theta with all equilibrium points substituted in should be 0,
# we get a value which is very close to 0 due to small rounding issues

theta_eq = theta.subs(PARAMETERS)
theta_eq

-232.913545280099 + 7.00714285714286*sin(41)

In [8]:
# Again, the value we get here is very close to 0. This can be accounted for through rounding
# errors throughout our calculations

I_dot_eq = I_dot.subs(PARAMETERS)
I_dot_eq

-6.54836185276508e-11

In [9]:
# Determine partial derivatives for Taylor's theorem

A = d_theta_x1 = theta.diff(x1)
B = d_theta_x2 = theta.diff(x2)
C = d_theta_x3 = theta.diff(x3)

D = d_I_dot_x1 = I_dot.diff(x1)
E = d_I_dot_x3 = I_dot.diff(x3)
F = d_I_dot_V = I_dot.diff(V)

In [10]:
# theta_eq subsituted for 0 here to simplify equation
theta_lin = 0 + A*(x1 - x1_sp) + B*(x2 - x2_sp) + C*(x3 - x3_sp)
theta_lin 

-5*b*x2/(7*m) + 10*c*x3*(x3 - 0.707488266645418)/(7*m*(delta - x1)**2) + 5*(x1 - 0.5)*(2*c*x3**2/(delta - x1)**3 - k)/(7*m)

In [11]:
# I_dot_eq subsituted for 0 here to simplify equation
I_dot_lin = 0 + D*(x1 - x1_sp) + E*(x3 - x3_sp) +  F*(V - V_sp)   
I_dot_lin 

-L_1*alpha*(x1 - 0.5)*(-R*x3 + V)*exp(-alpha*(delta - x1))/(L_0 + L_1*exp(-alpha*(delta - x1)))**2 - R*(x3 - 0.707488266645418)/(L_0 + L_1*exp(-alpha*(delta - x1))) + (V - 1556.47418661991)/(L_0 + L_1*exp(-alpha*(delta - x1)))

In [15]:
# Determine constant values for partial derivatives
A = d_theta_x1.subs(PARAMETERS)
B = d_theta_x2.subs(PARAMETERS)
C = d_theta_x3.subs(PARAMETERS)

D = d_I_dot_x1.subs(PARAMETERS)
E = d_I_dot_x3.subs(PARAMETERS)
F = d_I_dot_V.subs(PARAMETERS)

In [22]:
F

6.89037357307828